In [1]:
import st_acn
import os

In [2]:
import mnmstpy as mnmst

In [3]:
import scanpy as sc

section_id = '151675'
input_dir = os.path.join('F:\\UGIMC\\Data', section_id)
adata = sc.read_visium(path = input_dir, count_file = section_id+'_filtered_feature_bc_matrix.h5')
adata.var_names_make_unique()
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=3000)
hvg_filter = adata.var['highly_variable']
sc.pp.normalize_total(adata, inplace=True)
adata = adata[:, hvg_filter]
display(adata)

C:\Users\16419\Anaconda3\envs\your_out_env\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


View of AnnData object with n_obs × n_vars = 3592 × 3000
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'spatial', 'hvg'
    obsm: 'spatial'

In [4]:
enhanced_adata, cell_spatial = mnmst.data_enhance(adata, k_nei=6)


Median distance to closest cell = 138.00362314084367
num_nbrs: 3000


C:\Users\16419\Anaconda3\envs\your_out_env\lib\site-packages\mnmstpy\utils.py:181: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return anndata.AnnData(matrix, obs=adata.obs, var=var_combined, uns=adata.uns, obsm=adata.obsm)


In [5]:
sc.pp.pca(enhanced_adata)


In [6]:
low_dim_x

array([[-3.2056735 ,  3.5576563 ,  1.8095381 , ...,  0.03319357,
         1.7386935 , -0.79040796],
       [-6.0986304 , -2.4624887 , -1.5821352 , ..., -0.1268688 ,
         1.0101007 , -1.2005839 ],
       [24.870382  ,  0.10176751, -2.7089322 , ..., -1.0544227 ,
        -1.6599933 ,  0.4441213 ],
       ...,
       [24.85237   , -1.3490579 , -1.6128094 , ..., -0.40623134,
         3.4623342 , -1.3783656 ],
       [-3.5409114 , -1.3799828 ,  4.518481  , ..., -3.4852898 ,
        -0.9562641 ,  1.1378083 ],
       [12.777481  , -0.6544561 ,  0.55920327, ..., -1.2540019 ,
        -2.7717128 , -0.11662232]], dtype=float32)

In [7]:
cell_spatial.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
import pandas as pd
Ann_df = pd.read_csv(os.path.join('F:\\UGIMC\\Data', section_id, section_id+'_truth.txt'), sep='\t', header=None, index_col=0)
Ann_df.columns = ['Ground Truth']
adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth']
gt = pd.Categorical(adata.obs['Ground Truth']).codes

C:\Users\16419\AppData\Local\Temp\ipykernel_24652\1545262636.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'Ground Truth']


In [9]:
cell_spatial.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
low_dim_x

array([[-3.2056735 ,  3.5576563 ,  1.8095381 , ...,  0.03319357,
         1.7386935 , -0.79040796],
       [-6.0986304 , -2.4624887 , -1.5821352 , ..., -0.1268688 ,
         1.0101007 , -1.2005839 ],
       [24.870382  ,  0.10176751, -2.7089322 , ..., -1.0544227 ,
        -1.6599933 ,  0.4441213 ],
       ...,
       [24.85237   , -1.3490579 , -1.6128094 , ..., -0.40623134,
         3.4623342 , -1.3783656 ],
       [-3.5409114 , -1.3799828 ,  4.518481  , ..., -3.4852898 ,
        -0.9562641 ,  1.1378083 ],
       [12.777481  , -0.6544561 ,  0.55920327, ..., -1.2540019 ,
        -2.7717128 , -0.11662232]], dtype=float32)

In [11]:
import stACN
Z = stACN.stACN_master(low_dim_x, cell_spatial.A,gt)

(3592, 50)
(3592, 3592)


ValueError: setting an array element with a sequence.

In [ ]:
learned_graph_from_matlab = Z
import igraph as ig
import leidenalg
from natsort import natsorted
sources, targets = learned_graph_from_matlab.nonzero()
ans_weight = learned_graph_from_matlab[sources, targets]
g = ig.Graph(directed=True)
g.add_vertices(learned_graph_from_matlab.shape[0])  # this adds adjacency.shape[0] vertices
g.add_edges(list(zip(sources, targets)))
g.es['weight'] = ans_weight
partition_type = leidenalg.RBConfigurationVertexPartition
# clustering proper
partition_kwargs = {'weights': np.array(g.es['weight']).astype(np.float64), 'n_iterations': -1, 'seed': 42,'resolution_parameter': 1.16}
part = leidenalg.find_partition(g, partition_type, **partition_kwargs)
# store output into adata.obs
groups = np.array(part.membership)
leiden_label = pd.Categorical(
    values=groups.astype('U'),
    categories=natsorted(map(str, np.unique(groups))),
)
print(leiden_label)
adjusted_rand_score(pd.Categorical(adata.obs['Ground Truth']).codes, leiden_label)

In [ ]:
adata.obs['stACN'] = leiden_label
from utils.cluster import refine
refined_pred = refine(sample_id=adata.obs.index.tolist(), pred=adata.obs['stACN'].tolist(),dis=weights_adj, shape="hexagon")
adata.obs["stACN_refined_pred"] = refined_pred
adata.obs["stACN_refined_pred"] = adata.obs["stACN_refined_pred"].astype('category')
multi_ari = adjusted_rand_score(pd.Categorical(adata.obs['Ground Truth']).codes, adata.obs['MULTI_LABEL'])
refine_ari = adjusted_rand_score(pd.Categorical(adata.obs['Ground Truth']).codes, adata.obs['refined_pred'])
sc.pl.spatial(adata, color=['stACN', 'stACN_refined_pred', 'Ground Truth'], title=['stACN (ARI=%.2f)'% multi_ari, 'refine_multi (ARI=%.2f)'%refine_ari, 'Ground Truth'], alpha_img=0, size=1.6,save=section_id)